mmdet3d结果可视化

In [8]:
import numpy as np
import json
import cv2
import open3d as o3d

In [9]:
def get_3d_box_corners(bboxes):
    corners_3d_all = []
    for bbox in bboxes:
        x, y, z, l, w, h, yaw = bbox
        
        # 根据长宽高得到8个角点的相对坐标
        x_corners = l/2 * np.array([1, 1, -1, -1, 1, 1, -1, -1])  
        y_corners = w/2 * np.array([-1, 1, 1, -1, -1, 1, 1, -1])
        z_corners = h/2 * np.array([-1, -1, -1, -1, 1, 1, 1, 1])
        
        # 根据旋转角度旋转角点
        corners_3d = np.vstack((x_corners, y_corners, z_corners))
        c, s = np.cos(yaw), np.sin(yaw)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        corners_3d = np.dot(R, corners_3d)
        
        # 加上bbox中心坐标
        x_corners, y_corners, z_corners = corners_3d[0]+x, corners_3d[1]+y, corners_3d[2]+(z/2)
        corners_3d = np.vstack((x_corners, y_corners, z_corners))
        corners_3d_all.append(corners_3d)
        
    return corners_3d_all


In [10]:
def get_bbox3d(pcd_path,corners):
    pcd = pcd_path
    pcd = o3d.io.read_point_cloud(pcd)

    lidar_bbox3d = []
    for box3d in corners:
        lidar_box3d = np.transpose(box3d)
        lidar_bbox3d.append(lidar_box3d)

    lidar_3dboxes = np.array(lidar_bbox3d)
    return lidar_3dboxes,pcd

In [11]:
def draw_3D_box2point(lidar_3dboxes,points):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(points)
    num = len(lidar_3dboxes)
    for n in range(num):
        lidar_3dbox = lidar_3dboxes[n]

        # 创建Open3D点云
        lines_box = np.array([[0, 1], [1, 2], [2, 3],[0, 3], [4, 5], [5, 6],
                                [6, 7], [4, 7], [0, 4], [1, 5], [2, 6], [3, 7]]) 
        colors = np.array([[1, 0, 0] for j in range(len(lines_box))])

        # 创建Open3D线集
        line_set = o3d.geometry.LineSet() #创建line对象
        #将八个顶点转换成o3d可以使用的数据类型
        line_set.lines = o3d.utility.Vector2iVector(lines_box) 
        line_set.colors = o3d.utility.Vector3dVector(colors)  #设置每条线段的颜色
        #把八个顶点的空间信息转换成o3d可以使用的数据类型
        line_set.points = o3d.utility.Vector3dVector(lidar_3dbox) 
        vis.add_geometry(line_set)  #将矩形框加入到窗口中

        # pcd.paint_uniform_color([1, 0, 0])
    # 可视化点云
    vis.run()
    vis.destroy_window()

In [12]:
def get_calib(calib_path):
    with open(calib_path,'r') as f:
        calib = f.readlines()

    P2 = np.array([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3, 4)
    K = np.array(P2[:12]).reshape(3, 4)[:3, :3]

    # 从字符串解析出从激光雷达到相机的变换矩阵
    Tr_velo_to_cam = np.array([float(x) for x in calib[5].strip('\n').split(' ')[1:]]).reshape(3, 4)
    Tr_velo_to_cam = np.insert(Tr_velo_to_cam, 3, values=[0, 0, 0, 1], axis=0)
    Tr = np.array([Tr_velo_to_cam[0,3],Tr_velo_to_cam[1,3],Tr_velo_to_cam[2,3]])

    R0_rect = np.array([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3, 3)
    # R0_rect = np.insert(R0_rect, 3, values=[0, 0, 0], axis=0)
    # R0_rect = np.insert(R0_rect, 3, values=[0, 0, 0, 1], axis=1)


    # 输出相机内参矩阵和外参矩阵
    print("P2:")
    print(P2)
    print('K:')
    print(K)
    print('Tr_velo2cam:')
    print(Tr_velo_to_cam)
    print('T:')
    print(Tr)
    print('R0_rect:')
    print(R0_rect)
    return P2,K,Tr_velo_to_cam,Tr,R0_rect


In [13]:
def draw_3Dtoimg(img_path,corners,Tr_velo_to_cam,R0_rect,P2):
    num = len(corners)
    all_points_img = []
    for i in range(num):
        points_cam = np.append(corners[i].T, np.ones((8,1)), axis=1)
        points_cam = np.dot(Tr_velo_to_cam, points_cam.T).T
        points_cam = points_cam[:, :3]

        points_rect = np.dot(points_cam, np.transpose(R0_rect))

        # 投影到图像像素坐标
        fx = P2[0,0] 
        fy = P2[1,1]
        cx = P2[0,2]
        cy = P2[1,2]

        points_img = np.zeros((8,2))
        for i in range(8):
            x = points_rect[i,0]
            y = points_rect[i,1]
            z = points_rect[i,2]
            points_img[i,0] = x*fx/z + cx
            points_img[i,1] = y*fy/z + cy
            all_points_img.append(points_img)

    # 读取图像
    img = cv2.imread(img_path)
    print(all_points_img)
    # 将投影点坐标转为整数
    all_points_img = np.int0(all_points_img)

    lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0],  # 下面的4条线
                    [4, 5], [5, 6], [6, 7], [7, 4],  # 上面的4条线
                    [0, 4], [1, 5], [2, 6], [3, 7]]) # 连接上下面的4条线

        # 在图像上绘制3D框的12条线
    for points_img in all_points_img:
        for start, end in lines:
            start_x, start_y = points_img[start]
            end_x, end_y = points_img[end]
            img = cv2.line(img, (start_x, start_y), (end_x, end_y), (0, 0, 255), 2)

    # 放大图像
    scale_factor = 2  # 设置缩放因子,2表示放大两倍
    new_width = int(img.shape[1] * scale_factor)
    new_height = int(img.shape[0] * scale_factor)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    # 显示修改后的图像
    cv2.imshow('3D Bounding Box', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [14]:
with open('000008.json' , 'r') as f:
    data = json.load(f)
# 解析JSON数据
labels_3d = data['labels_3d']
scores_3d = data['scores_3d']
bboxes_3d = data['bboxes_3d']
img_path='000008.png'
calib_path = '000008.txt'
pcd_path = 'pcd_nongrond_seg.pcd'

corners=get_3d_box_corners(bboxes_3d)
lidar_bbox3d,pcd=get_bbox3d(pcd_path,corners)
P2,K,Tr_velo_to_cam,Tr,R0_rect = get_calib(calib_path)
draw_3D_box2point(lidar_bbox3d,pcd)
draw_3Dtoimg(img_path,corners,Tr_velo_to_cam,R0_rect,P2)

P2:
[[7.215377e+02 0.000000e+00 6.095593e+02 4.485728e+01]
 [0.000000e+00 7.215377e+02 1.728540e+02 2.163791e-01]
 [0.000000e+00 0.000000e+00 1.000000e+00 2.745884e-03]]
K:
[[721.5377   0.     609.5593]
 [  0.     721.5377 172.854 ]
 [  0.       0.       1.    ]]
Tr_velo2cam:
[[ 7.533745e-03 -9.999714e-01 -6.166020e-04 -4.069766e-03]
 [ 1.480249e-02  7.280733e-04 -9.998902e-01 -7.631618e-02]
 [ 9.998621e-01  7.523790e-03  1.480755e-02 -2.717806e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
T:
[-0.00406977 -0.07631618 -0.2717806 ]
R0_rect:
[[ 0.9999239   0.00983776 -0.00744505]
 [-0.0098698   0.9999421  -0.00427846]
 [ 0.00740253  0.00435161  0.9999631 ]]
[array([[718.34535442, 246.2566967 ],
       [648.29051523, 244.98766284],
       [594.39090406, 263.52042843],
       [682.43135828, 265.89156137],
       [717.49335619, 176.00423078],
       [647.53312298, 176.84072283],
       [593.4922914 , 176.54113541],
       [681.38184632, 175.45268549]]), array([[718.3453544

C:\Users\6666\AppData\Local\Temp\ipykernel_8588\2997559079.py:30: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  all_points_img = np.int0(all_points_img)
